# Notebook 4: Realtime Recommendations

Specify "Python 3" Kernel  and "Data Science" Image. Set the instance type as ml.t3.medium (default) for this notebook.

### Background

In this notebook, we will get personalized product recommendations using the Collaborative Filtering model and the Ranking model we deployed in notebook 2.

We'll do this by:

1. Using the deployed Collborative Filtering model which will take the payload that we cached in notebook 2 (to address the cold start problem) as input and generate candidate products based on the user-item interaction.
2. Feeding these candidate products into our Ranking model along with click stream data from our `click stream` Feature Group. You'll see that the real-time click stream data will directly influence the ranking of the candidate products at the time of inference.

<img src="./img/inference-arch.png" alt="Inference arch" style="width: 800px;"/>

### Package updates

In [40]:
!pip install --upgrade pandas

### Imports

In [41]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import numpy as np
import pandas as pd
from utils import *
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import random
from parameter_store import ParameterStore
import time

### Session variables

In [42]:
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
featurestore_runtime = boto3.client(service_name='sagemaker-featurestore-runtime', region_name=region)
ps = ParameterStore(verbose=False)
ps.set_namespace('feature-store-workshop')

In [43]:
query_results= 'sagemaker-recsys-featurestore-workshop'
prefix = 'recsys-feature-store'

Load variables from previous notebooks.

In [45]:
parameters = ps.read()
customers_feature_group_name = parameters['customers_feature_group_name']
products_feature_group_name = parameters['products_feature_group_name']
orders_feature_group_name = parameters['orders_feature_group_name']
click_stream_historical_feature_group_name = parameters['click_stream_historical_feature_group_name']
click_stream_feature_group_name = parameters['click_stream_feature_group_name']

customers_table = parameters['customers_table']
products_table = parameters['products_table']
orders_table = parameters['orders_table']
click_stream_historical_table = parameters['click_stream_historical_table']
click_stream_table = parameters['click_stream_table']

train_data_location = parameters['train_data_location']
test_data_location = parameters['test_data_location']

cf_model_endpoint_name = parameters['cf_model_endpoint_name']
ranking_model_endpoint_name = parameters['ranking_model_endpoint_name']

customer_id = parameters['inference_customer_id']

%store -r

### Get unranked recommended products from the Collaborative Filtering model

Create a `Predictor` object from our collaborative filtering model endpoint (which we deployed in notebook 2) so that we can use it to make predictions.

In [46]:
# Make sure model has finished deploying
existing_endpoints = sagemaker_session.sagemaker_client.list_endpoints(NameContains=cf_model_endpoint_name, MaxResults=30)["Endpoints"]
while not existing_endpoints:
    time.sleep(60)
    existing_endpoints = sagemaker_session.sagemaker_client.list_endpoints(NameContains=cf_model_endpoint_name, MaxResults=30)["Endpoints"]

cf_model_predictor = sagemaker.predictor.Predictor(endpoint_name=cf_model_endpoint_name, 
                                                   sagemaker_session=sagemaker_session,
                                                   serializer=FMSerializer(),
                                                   deserializer=JSONDeserializer())

In [47]:
# Pass in our cached data as input to the Collaborative Filtering model
predictions = cf_model_predictor.predict(cf_inference_payload)['predictions']

# Add those predictions to the input DataFrame
predictions = [prediction["score"] for prediction in predictions]
cf_inference_df['predictions'] = predictions

# Sort by predictions and take top 10
cf_inference_df = cf_inference_df.sort_values(by='predictions', ascending=False).head(10).reset_index()

Let's take a look at the output from our collaborative filtering model. These unranked products are recommended to the customer based on their purchase history.

In [48]:
cf_inference_df

,index,customer_id,product_id,state,age,is_married,product_name,predictions
0,6,C3571,P6176,maine,35,0,"pretzel \""shells\""",1.723941
1,8,C3571,P11086,maine,35,0,almonds mini nut-thins cheddar cheese,1.670082
2,2,C3571,P16823,maine,35,0,pirouette chocolate fudge creme filled wafers,1.640127
3,12,C3571,P15430,maine,35,0,organic pork chop seasoning,1.639393
4,5,C3571,P6247,maine,35,0,fruit punch roarin' waters,1.587156
5,7,C3571,P14539,maine,35,0,"chocolate, organic, unsweetened, 100% cacao",1.505779
6,1,C3571,P10682,maine,35,0,mini cakes birthday cake,1.501803
7,4,C3571,P4152,maine,35,0,white cheddar bunnies,1.464058
8,13,C3571,P7822,maine,35,0,degreaser,1.454165
9,3,C3571,P5429,maine,35,0,organic sweet potato puree,1.414727


These products aren't personalized which is what we'll use the ranking model for. The ranking model will take into account the customer's current behavior on the website which will influence the ranking of the recommended products.

### Rank the recommended products using the Ranking model

Create a `Predictor` object from our ranking model endpoint (which we deployed in notebook 2) so that we can use it to make predictions.

In [49]:
# Make sure model has finished deploying
existing_endpoints = sagemaker_session.sagemaker_client.list_endpoints(NameContains=ranking_model_endpoint_name, MaxResults=30)["Endpoints"]
while not existing_endpoints:
    time.sleep(60)
    existing_endpoints = sagemaker_session.sagemaker_client.list_endpoints(NameContains=ranking_model_endpoint_name, MaxResults=30)["Endpoints"]

ranking_model_predictor = sagemaker.predictor.Predictor(endpoint_name=ranking_model_endpoint_name, 
                                                        sagemaker_session=sagemaker_session,
                                                        serializer = CSVSerializer())

In order to construct the input for the ranking model, we need to one-hot encode product categories as we did in training. Ideally, these one-hot encoded products would be cached somewhere (perhaps in SageMaker Feature Store or another AWS service) but we're keeping it simple for now.

In [50]:
query = f'''
select product_category
from "{products_table}"
order by product_category
'''
product_categories_df, query = query_offline_store(products_feature_group_name, query,
                                                   sagemaker_session)
one_hot_cat_features = product_categories_df.product_category.unique()

df_one_hot_cat_features = pd.DataFrame(one_hot_cat_features)
df_one_hot_cat_features.columns = ['product_category']

df_one_hot_cat_features = pd.concat([df_one_hot_cat_features, pd.get_dummies(df_one_hot_cat_features['product_category'], prefix='cat')],axis=1)

# Convert dummy variables to integers
for col in df_one_hot_cat_features.columns[1:]:
    df_one_hot_cat_features[col] = df_one_hot_cat_features[col].astype(int)

Now we create a function to take the output from the collaborative filtering model and join it with the one-hot encoded product categories _**AND**_ the real-time click stream data from our `click stream` Feature Group, as this data will influence the ranking of recommended products. You can imagine the ranking inputs like so:

<img src="./img/vector-input-for-ranking.png" alt="Ranking input" style="width: 600px;"/>


We'll use this function in the next section.

In [51]:
def get_ranking_model_input_data(df, df_one_hot_cat_features):
    product_category_list = []
    product_health_index_list = []
    
    customer_id = df.iloc[0]['customer_id']
    # Get customer features from customers_feature_group_name
    customer_record = featurestore_runtime.get_record(FeatureGroupName=customers_feature_group_name,
                                                      RecordIdentifierValueAsString=customer_id,
                                                      FeatureNames=['customer_health_index'])
    print(customer_record)
    customer_health_index = customer_record['Record'][0]['ValueAsString']
    
    # Get product features (instead of looping, you can optionally use
    # the `batch_get_record` Feature Store API)
    for index, row_tuple in df.iterrows():
        
        product_id = row_tuple['product_id']
        
        # Get product features from products_feature_group_name
        product_record = featurestore_runtime.get_record(FeatureGroupName=products_feature_group_name,
                                                         RecordIdentifierValueAsString=product_id,
                                                         FeatureNames=['product_category',
                                                                       'product_health_index'])
        
        product_category = product_record['Record'][0]['ValueAsString']
        product_health_index = product_record['Record'][1]['ValueAsString']
        
        product_category_list.append(product_category)
        product_health_index_list.append(product_health_index)

        

    # Get click stream features from customers_click_stream_feature_group_name
    click_stream_record = featurestore_runtime.get_record(FeatureGroupName=click_stream_feature_group_name,
                                                          RecordIdentifierValueAsString=customer_id,
                                                          FeatureNames=['sum_activity_weight_last_2m',
                                                                  'avg_product_health_index_last_2m'])
    print(click_stream_record)
    
    # Calculate healthy_activity_last_2m as this will influence ranking as well
    sum_activity_weight_last_2m = click_stream_record['Record'][0]['ValueAsString']
    avg_product_health_index_last_2m = click_stream_record['Record'][1]['ValueAsString']
    healthy_activity_last_2m = int(sum_activity_weight_last_2m) * float(avg_product_health_index_last_2m)

    data = {'healthy_activity_last_2m': healthy_activity_last_2m,
            'product_health_index': product_health_index_list,
            'customer_health_index': customer_health_index,
            'product_category': product_category_list}
    
    ranking_inference_df = pd.DataFrame(data)
    ranking_inference_df = ranking_inference_df.merge(df_one_hot_cat_features, on='product_category',
                                                      how='left')
    del ranking_inference_df['product_category']

    return ranking_inference_df

### Real-time personalized product recommendations

Let's finally put everything together by calling the function we created above to get real-time personalized product recommendations using data that's being streamed to SageMaker Feature Store to influence ranking.

Remember that we already have our unranked recommended products for our collaborative filtering model.

In [52]:
cf_inference_df

,index,customer_id,product_id,state,age,is_married,product_name,predictions
0,6,C3571,P6176,maine,35,0,"pretzel \""shells\""",1.723941
1,8,C3571,P11086,maine,35,0,almonds mini nut-thins cheddar cheese,1.670082
2,2,C3571,P16823,maine,35,0,pirouette chocolate fudge creme filled wafers,1.640127
3,12,C3571,P15430,maine,35,0,organic pork chop seasoning,1.639393
4,5,C3571,P6247,maine,35,0,fruit punch roarin' waters,1.587156
5,7,C3571,P14539,maine,35,0,"chocolate, organic, unsweetened, 100% cacao",1.505779
6,1,C3571,P10682,maine,35,0,mini cakes birthday cake,1.501803
7,4,C3571,P4152,maine,35,0,white cheddar bunnies,1.464058
8,13,C3571,P7822,maine,35,0,degreaser,1.454165
9,3,C3571,P5429,maine,35,0,organic sweet potato puree,1.414727


Now, we need to rank those products recommendations.

In [53]:
# Construct input data for the ranking model
ranking_inference_df = get_ranking_model_input_data(cf_inference_df, df_one_hot_cat_features)

{'ResponseMetadata': {'RequestId': 'a8a788d0-13d3-4a21-9e04-5fd505a2cf3a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'a8a788d0-13d3-4a21-9e04-5fd505a2cf3a', 'content-type': 'application/json', 'content-length': '131', 'date': 'Wed, 29 May 2024 00:49:52 GMT'}, 'RetryAttempts': 0}, 'Record': [{'FeatureName': 'customer_health_index', 'ValueAsString': '0.0952770902420399'}]}
{'ResponseMetadata': {'RequestId': 'b80bfb05-4a85-46a2-9e3a-97698adfd24d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b80bfb05-4a85-46a2-9e3a-97698adfd24d', 'content-type': 'application/json', 'content-length': '234', 'date': 'Wed, 29 May 2024 00:49:53 GMT'}, 'RetryAttempts': 0}, 'Record': [{'FeatureName': 'sum_activity_weight_last_2m', 'ValueAsString': '9'}, {'FeatureName': 'avg_product_health_index_last_2m', 'ValueAsString': '0.23333333333333336'}]}


In [54]:
# Get our ranked product recommendations and attach the predictions to the model input
# ranking_inference_df['propensity_to_buy'] = ranking_model_predictor.predict(ranking_inference_df.to_numpy()).decode('utf-8').split('\n')
predictions = ranking_model_predictor.predict(ranking_inference_df.to_numpy()).decode('utf-8').split('\n')
predictions = [float(p) for p in predictions if p != '']  # Convert to float and remove empty strings

ranking_inference_df['propensity_to_buy'] = predictions


Now that we have our personalized ranked recommendations, let's see what the top 5 recommended products are.

In [55]:
# Join all the data back together for inspection
personalized_recommendations = pd.concat([cf_inference_df[['customer_id', 'product_id', 'product_name']],
                                          ranking_inference_df[['propensity_to_buy']]], axis=1)

# And sort by propensity to buy
personalized_recommendations.sort_values(by='propensity_to_buy', ascending=False)[['product_id','product_name', 'propensity_to_buy']].reset_index(drop=True).head(10)

,product_id,product_name,propensity_to_buy
0,P6176,"pretzel \""shells\""",0.999014
1,P11086,almonds mini nut-thins cheddar cheese,0.999014
2,P16823,pirouette chocolate fudge creme filled wafers,0.999014
3,P6247,fruit punch roarin' waters,0.999014
4,P4152,white cheddar bunnies,0.999014
5,P10682,mini cakes birthday cake,0.999014
6,P5429,organic sweet potato puree,0.998438
7,P14539,"chocolate, organic, unsweetened, 100% cacao",0.998438
8,P15430,organic pork chop seasoning,0.439611
9,P7822,degreaser,0.439611


<div class="alert alert-info"> 💡 <strong> Personalized recommendations </strong>
Note how the ranking changed based on the customer's last 2 minutes of activity on the website.
</div>

### Congratulations! 🎉

You've officially built a recommendation engine system leveraging SageMaker Feature Store as a way to both train the recommendation engine models and influence recommendations in real-time! 🎉 💪

### (Optional)

Now you've seen real-time recommendations for a customer interacting with unhealthy products, you can repeat this experiment to simulate healthy product interactions by ingesting new data to the stream. If you'd like to do this, follow the below steps:

1. Go to notebook 3, navigate to the last section.
2. Replace `put_records_in_kinesis_stream(inference_customer_id, 0.1, 0.3)` with `put_records_in_kinesis_stream(inference_customer_id, 0.7, 0.9)`.
3. Wait for the data to be ingested to the Kinesis Data Stream.
4. Re-run the "Real-time personalized recommendation" section in this notebook.

Go back to Workshop Studio and click on "Next".